In [ ]:
!pip install tensorflow nltk
!pip install nltk
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 860.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 154.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta 0:00:00


In [9]:
import nltk
nltk.download('all')
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
import random
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pickle
import nltk

In [4]:


intents = json.loads(open('intents.json').read())


In [13]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in document[0]]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Save words and classes
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Build model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile and train
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print("🎉 Model training complete!")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0508 - loss: 2.6039    
Epoch 2/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2232 - loss: 2.4634 
Epoch 3/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1180 - loss: 2.4530     
Epoch 4/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2596 - loss: 2.2985     
Epoch 5/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2287 - loss: 2.3459 
Epoch 6/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3660 - loss: 2.1308 
Epoch 7/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3801 - loss: 2.0194 
Epoch 8/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4726 - loss: 1.7585 
Epoch 9/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4088 - loss: 1.8070 
Epoch 10/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4601 - loss: 1.6857     
Epoch 11/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6532 - loss: 1.4611 
Epoch 12/200
19/19 ━━━━━━━━━━━━━━━━━━

🎉 Model training complete!


In [16]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    if len(ints) == 0:
        tag = "noanswer"
    else:
        tag = ints[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])

def chatbot_response(text):
    ints = predict_class(text)
    return get_response(ints, intents)

# Chat loop
print("🤖 Chatbot ready! Type 'quit' to stop.")
while True:
    msg = input("You: ")
    if msg.lower() == "quit":
        print("Bot: Goodbye 👋")
        break
    res = chatbot_response(msg)
    print("Bot:", res)


🤖 Chatbot ready! Type 'quit' to stop.
You: delivery kitny dino m hogi?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Bot: You can also choose express delivery for faster shipping.
You: how many days will it take to deliver order?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Bot: Product must be unused, unworn, and returned in its original packaging along with all tags. Returns will not be accepted for used, washed, or damaged items. In case a damaged or wrong item is received, please record an unedited and unfiltered unboxing video as proof and share it with our support team within 24 hours for a smooth return or exchange process.
You: delivery charges?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Bot: We usually deliver within 3 to 5 business days. 🚚
You: kia delivery charges advance pay krna hon gy?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Bot: We usually deliver within 3 to 5 business days. 🚚
You: how can i return my parcel?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Bot: You can return or exchange items within 

In [18]:
# Zaroori Libraries Import Karen
import gradio as gr
import random
import numpy as np
import json
import nltk
from nltk.stem import WordNetLemmatizer
# 👇 Yeh line zaroor shamil karein
from tensorflow.keras.models import load_model
# -----------------------------------------------
import pickle
import time

lemmatizer = WordNetLemmatizer()
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
print("✅ Files aur Model kamyabi se load ho gaye hain!")


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    p = bow(sentence, words)
    # Model ko input dene se pehle shape (1, features) mein hona chahiye
    res = model.predict(np.array([p]), verbose=0)[0]

    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    if len(ints) == 0:
        # Agar koi intent na mile, toh 'noanswer' tag ka response dein
        tag = "noanswer"
        for i in intents_json['intents']:
            if i['tag'] == tag:
                 return random.choice(i['responses'])
        return "Maaf kijiye, main aapki baat samajh nahi paya." # Default fallback
    else:
        tag = ints[0]['intent']
        for i in intents_json['intents']:
            if i['tag'] == tag:
                return random.choice(i['responses'])

# --- 5. Gradio Interface Function ---

# Yeh function Gradio ke input ko lega aur response generate karega.
def greet_and_respond(message, history):
    # Chatbot ka response generate karna
    response = get_response(predict_class(message), intents)

    # Thoda sa delay (optional, sirf behtar user experience ke liye)
    time.sleep(1)

    return response

# --- 6. Gradio Interface Define Karen Aur Launch Karen ---

# Interface components define karen

gr.ChatInterface(
    # Aapka response function
    fn=greet_and_respond,

    # Interface ka title aur description
    title="🛒 Customer Support Chatbot (TensorFlow)",
    description="Main aapke orders, products, aur technical sawalon ka jawab dene ke liye hazir hoon.",

    # Custom CSS (optional, thoda rang-roghan ke liye)
    css="""
        .gradio-container {background-color: #f0f8ff;}
        .message-wrap.user {background-color: #b0e0e6; color: black; border-radius: 15px;}
        .message-wrap.bot {background-color: #ffe4e1; color: black; border-radius: 15px;}
    """,

# Launch karen (Colab mein yeh ek public URL generate karega)
).launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


✅ Files aur Model kamyabi se load ho gaye hain!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://827c990516eb0e4daf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
